# Hellinger Fidelity and Distance Analysis

This notebook performs an analysis of Hellinger fidelity and distance for the Transverse Field Ising Model (TFIM) and Heisenberg Hamiltonians. The analysis leverages both classical exact calculations and quantum trotterization steps.

**To run the notebook correctly, run the cells in order from top-to-bottom.** 

The notebook produces four main plots:

1. **Hellinger Fidelity at Each Time Step**: Comparing the classical exact simulation to the trotterized simulation.
2. **Hellinger Distance from Initial State (Quantum Trotterization)**: Tracking the distance using quantum trotterization steps.
3. **Hellinger Distance from Initial State (Classical Calculation)**: Utilizing many small time steps for precise calculations.
4. **Combined Plot**: Overlaying the results from the quantum trotterization and classical calculations on the same axis for comparison.

Note that only plot 1 is directly relevant to benchmarking and is similar to method 2 in the primary benchmarking code.

Plots 2-4, while offering insight into state evolution, are not intended to serve as performance metrics. Instead, they are an example of a simple way to track state evolution against time. 

In [ ]:
import sys
import numpy as np
from qiskit_aer import Aer
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_aer.primitives import Sampler
from qiskit_algorithms import SciPyRealEvolver, TimeEvolutionProblem
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp, hellinger_distance, hellinger_fidelity
from qiskit.visualization import plot_distribution
import matplotlib.pyplot as plt

# Adjusting the path for custom imports
sys.path[1:1] = ["../_common", "../../_common/qiskit"]
import hamiltonian_simulation_kernel as ham
from hamiltonian_simulation_exact import HamiltonianSimulationExact



def get_probability_distribution(circuit, shots=None):
    """
    Generates the probability distribution of the outcomes for a given Qiskit circuit.

    Parameters:
    - circuit (QuantumCircuit): The quantum circuit to simulate.
    - shots (int): The number of shots for the simulation.

    Returns:
    - result (dict): The probability distribution of the outcomes.
    """

    sampler = Sampler()

    # shots = None means the results will be exact?
    result = sampler.run(circuit, shots=shots).result()

    dist = result.quasi_dists[0].binary_probabilities()

    return dist


def get_init_state_distribution(n_spins, hamiltonian):
    """
    Generates the initial state probability distribution based on the Hamiltonian.

    Parameters:
    - n_spins (int): Number of spins (qubits) in the system.
    - hamiltonian (str): Type of Hamiltonian (e.g., "tfim" for Transverse Field Ising Model).

    Returns:
    - dict: The probability distribution of the initial state.
    """
    if hamiltonian.lower().strip() == "tfim":
        init_state = "ghz"
    else:
        init_state = "checkerboard"

    init_circ = ham.initial_state(n_spins, init_state)

    init_circ.measure_all()

    return get_probability_distribution(init_circ)


def get_dist_at_timestep(step, K, max_time, hamiltonian, w, hx, hz):
    """
    Generates the probability distribution at a specific time step during the simulation.

    Parameters:
    - step (int): Current time step in the simulation.
    - K (int): Total number of time steps.
    - max_time (float): Maximum simulation time.
    - hamiltonian (str): Type of Hamiltonian (e.g., "tfim" for Transverse Field Ising Model).
    - w, hx, hz (float): Parameters for the Hamiltonian.

    Returns:
    - dict: The probability distribution at the given time step.
    """
    if step == 0:
        return get_init_state_distribution(n_spins=6, hamiltonian=hamiltonian)
    else:
        return get_probability_distribution(
            ham.HamiltonianSimulation(
                n_spins=6,
                K=step,
                t=step * (max_time / K),
                hamiltonian=hamiltonian,
                w=w,
                hx=hx,
                hz=hz,
            )
        )


In [ ]:
# produce four plots.
#
# Plot #1: Hellinger fidelity at each time step from the classical exact to the trotterized at each time step.
# Plot #2: Hellinger distance from initial state using quantum trotterization steps.
# Plot #3: Hellinger distance from initial state using exact classical calculation (can use many more little time steps.)
# Plot #4: Combine plots #1 and #2 onto the same axis.
# All four of these plots will be saved onto the computer into the working directory.

# These parameters define Heisenberg model variables.
np.random.seed(26)
hx = list(2 * np.random.random(20) - 1)  # random numbers between [-1, 1]
np.random.seed(75)
hz = list(2 * np.random.random(20) - 1)  # random numbers between [-1, 1]
w = 1


# Number of trotter steps 
K = 10

# Hamiltonians, along with a desired initial state. 
hamiltonians_initstate_pairs = [("heisenberg", "checkerboard"), ("tfim", "ghz")]

# The total amount of time to split into K trotter steps of size max_time / K. 
max_time = 0.4

In [ ]:
# Plot #1: Hellinger fidelity at each time step from the classical exact to the trotterized at each time step.

for hamiltonian, init_state in hamiltonians_initstate_pairs:
    hellingers = []
    times = []
    initial_distribution = None

    for k_i in range(K + 1):
        # note k_0 = 0
        dist = get_dist_at_timestep(k_i, K, max_time, hamiltonian, w, hx, hz)

        exact_dist = HamiltonianSimulationExact(
            6,
            t=k_i * (max_time / K),
            init_state=init_state,
            hamiltonian=hamiltonian,
            w=w,
            hx=hx,
            hz=hz,
        )

        hellingers.append(hellinger_fidelity(exact_dist, dist))
        times.append(k_i * (max_time / K))

    plt.plot(times, hellingers, marker="o")
plt.title("Hellinger fidelity of exact vs trotterized states at discrete times")
plt.legend(["heisenberg", "tfim"])
plt.xlabel("Time")
plt.ylabel("Hellinger fidelity")
plt.show()

In [ ]:
# Plot #2: Hellinger distance from initial state using quantum trotterization steps.

for hamiltonian, init_state in hamiltonians_initstate_pairs:

    hellingers = []
    times = []
    initial_distribution = None

    for k_i in range(K + 1):
        # note k_0 = 0
        #
        dist = get_dist_at_timestep(k_i, K, max_time, hamiltonian, w, hx, hz)

        if k_i == 0:
            first_dist = dist

        hellingers.append(hellinger_distance(first_dist, dist))
        times.append(k_i * (max_time / K))

    plt.plot(times, hellingers, marker="o")
plt.title("Hellinger distance from initial state VS time for quantum trotterization steps")
plt.legend(["trotterized heisenberg", "trotterized tfim"])
plt.xlabel("Time")
plt.ylabel("Hellinger distance")
plt.show()

In [ ]:
# Plot #3: Hellinger distance from initial state using exact classical calculation (can use many more little time steps.)

for hamiltonian, init_state in hamiltonians_initstate_pairs:
    hellinger_list = []
    first_exact = None

    for t in np.linspace(0, max_time, 100):

        exact_dist = HamiltonianSimulationExact(
            6, t, init_state=init_state, hamiltonian=hamiltonian, w=w, hx=hx, hz=hz
        )

        if t == 0:
            first_exact = exact_dist

        dist = hellinger_distance(first_exact, exact_dist)
        hellinger_list.append(dist)

    plt.plot([t for t in np.linspace(0, max_time, 100)], hellinger_list)
plt.title("Hellinger distance from initial state for exact evolution VS time")
plt.legend(["exact heisenberg", "exact tfim"])
plt.xlabel("Time")
plt.ylabel("Hellinger distance")
plt.show()

In [ ]:
# Plot #4: Combine plots #1 and #2 onto the same axis.

for hamiltonian, init_state in hamiltonians_initstate_pairs:

    hellingers = []
    times = []
    initial_distribution = None

    for k_i in range(K + 1):
        # note k_0 = 0

        dist = get_dist_at_timestep(k_i, K, max_time, hamiltonian, w, hx, hz)

        if k_i == 0:
            first_dist = dist

        hellingers.append(hellinger_distance(first_dist, dist))
        times.append(k_i * (max_time / K))

    plt.plot(times, hellingers, marker="o")

for hamiltonian, init_state in hamiltonians_initstate_pairs:
    hellinger_list = []
    first_exact = None

    for t in np.linspace(0, max_time, 100):

        exact_dist = HamiltonianSimulationExact(
            6, t, init_state=init_state, hamiltonian=hamiltonian, w=w, hx=hx, hz=hz
        )

        if t == 0:
            first_exact = exact_dist

        dist = hellinger_distance(first_exact, exact_dist)
        hellinger_list.append(dist)

    plt.plot([t for t in np.linspace(0, max_time, 100)], hellinger_list)
plt.title("Hellinger distance from initial state VS time")
plt.legend(
    ["trotterized heisenberg", "trotterized tfim", "exact heisenberg", "exact tfim"]
)
plt.xlabel("Time")
plt.ylabel("Hellinger distance")
plt.show()